In [1]:
import xarray as xr
import numpy  as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import metpy.calc as mpcalc
import climlab.utils.thermo as clab
from climlab.utils.constants import ps, kappa, tempCtoK, eps, Rd, Rv, cpv, cp, g, Lhvap, sigma

In [2]:
# calculate the ELF

def Gammam(T,p):
    #  Gammam  = -dtheta/dz is the rate of potential temperature decrease along the moist adiabat
    #  in K / m
    Gammam_value = (g/cp*(1.0 - (1.0 + Lhvap*clab.qsat(T,p) / Rd / T) /
                   (1.0 + Lhvap**2 * clab.qsat(T,p)/ cp/Rv/T**2)))
    return Gammam_value

def ELF(T0,T700,Q0):

    # Lower Tropospheric Stability (theta700 - theta0)
    LTS = clab.potential_temperature(T700, 700) - T0
    # Assume 80% relative humidity to compute LCL, appropriate for marine boundary layer
    LCL = clab.lifting_condensation_level(T0, 0.8)
    p_LCL = 1000.0*np.exp(-g*LCL/Rd/T0)
    T_LCL = T0 - 9.8*LCL/1000.  # 9.8 K/km, dry adiabatic rate

    # Assuming z_ML = z_LCL
    Gammam_DL = Gammam(T_LCL,p_LCL)

    z700 = (Rd*T0/g)*np.log(1000./700.)
    Gammam700 = Gammam(T700,700)

    # Park and Shin (2019), Eqn 4
    d_zs = 2750 # scale height, m
    zinv = -(LTS/Gammam700) + z700 + d_zs*(Gammam_DL/Gammam700)
    
    beta2 = np.sqrt(zinv*LCL)/d_zs
#    f     = Q0.apply(lambda x: np.max([0.15, np.min([1.0, x/0.003])]))
    f     = ((Q0/0.003).clip(upper=1.0)).clip(lower=0.15)
    ELF   = f*np.sqrt(1-beta2)

    return ELF
    

In [3]:
f = xr.open_dataset('~/work/LCC_pred/data/ERA5_SSF_monthly.nc')

lat      = f['lat']
lon      = f['lon']
time     = f['time']
lsm      = f['lsm']
LCF      = (f['cldarea_low_day_mon'] + f['cldarea_low_daynight_mon'])/2.0/100.0 # unit, 1
T1000    = f['t'].sel(level = 1000)
T850     = f['t'].sel(level = 850)
T700     = f['t'].sel(level = 700)
TH1000   = T1000
TH850    = T850*(1000/850)**(0.286)
TH700    = T700*(1000/700)**(0.286)
RH1000   = f['r'].sel(level = 1000)
RH850    = f['r'].sel(level = 850)
RH700    = f['r'].sel(level = 700)
Q1000    = f['q'].sel(level = 1000)
Q850     = f['q'].sel(level = 850)
Q700     = f['q'].sel(level = 700)
U        = np.sqrt(f['u']**2 + f['v']**2)
U1000    = U.sel(level = 1000)
U700     = U.sel(level = 700)
OMEGA500 = f['w'].sel(level = 500)
OMEGA700 = f['w'].sel(level = 700)
PWV      = f['tcwv']
LH       = f['slhf']
SH       = f['sshf']
SST      = f['sst']
AOD      = f['modis_dtdb_aod55_mon']

u_wind = f['u'].sel(level = 1000)
v_wind = f['v'].sel(level = 1000)
Tadv   = mpcalc.advection(SST, u_wind, v_wind)

t_new, lat_new, lon_new = np.meshgrid(time, lat, lon, indexing='ij')

data = {'time':np.ndarray.flatten(t_new),
        'lat':np.ndarray.flatten(lat_new),
        'lon':np.ndarray.flatten(lon_new),
        'lsm':lsm.values.flatten(),
        'LCF': LCF.values.flatten(),          # 1
        'T1000':T1000.values.flatten(),       # K
        'T700':T700.values.flatten(),         # K
        'TH1000':TH1000.values.flatten(),     # K
        'TH850':TH850.values.flatten(),       # K
        'TH700':TH700.values.flatten(),       # K
        'RH1000':RH1000.values.flatten(),     # %
        'RH850':RH850.values.flatten(),       # %
        'RH700':RH700.values.flatten(),       # %
        'Q1000':Q1000.values.flatten(),       # kg/kg
        'Q850':Q850.values.flatten(),         # kg/kg
        'Q700':Q700.values.flatten(),         # kg/kg
        'U1000':U1000.values.flatten(),       # m/s
        'U700':U700.values.flatten(),         # m/s
        'OMEGA500':OMEGA500.values.flatten(), # Pa/s
        'OMEGA700':OMEGA700.values.flatten(), # Pa/s
        'PWV':PWV.values.flatten(),           # kg/m2
        #The energy (turbulent and radiative) and momentum fluxes should be divided by 86400 seconds (24 hours) to convert to the commonly used units of Wm-2 and Nm-2, respectively.
        # https://confluence.ecmwf.int/display/CKB/ERA5%3A+data+documentation#ERA5:datadocumentation-Meanrates/fluxesandaccumulations
        'LH':LH.values.flatten()/86400,       # J/m2 to W/m2
        'SH':SH.values.flatten()/86400,       # j/m2 to W/m2
        'SST':SST.values.flatten(),           # K
        'Tadv':Tadv.values.flatten()*3600*24, # K/day
        'AOD':AOD.values.flatten()
       }

df = pd.DataFrame(data)
df = df.set_index('time')

df['dQ']    = df.Q1000 - df.Q700
df['LTS']   = df.TH700 - df.TH1000
df['EIS']   = clab.EIS(df.T1000, df.T700)
df['ECTEI'] = df.EIS - 0.23*Lhvap/cp*df.dQ
df['ELF']   = ELF(df.T1000, df.T700, df.Q1000)

/glade/work/hpzhang/tools/anaconda3/envs/geocat/lib/python3.9/site-packages/metpy/xarray.py:1471: UserWarning: Vertical dimension number not found. Defaulting to (..., Z, Y, X) order.
  warnings.warn(
/glade/work/hpzhang/tools/anaconda3/envs/geocat/lib/python3.9/site-packages/pandas/core/arraylike.py:397: RuntimeWarning: invalid value encountered in sqrt
  result = getattr(ufunc, method)(*inputs, **kwargs)


In [4]:
df.head()

,lat,lon,lsm,LCF,T1000,T700,TH1000,TH850,TH700,RH1000,...,LH,SH,SST,Tadv,AOD,dQ,LTS,EIS,ECTEI,ELF
time,,,,,,,,,,,,,,,,,,,,,
2003-01-15,-59.5,0.5,0.0,0.590822,273.145630,259.122681,273.145630,277.822327,286.950775,85.577278,...,-14.891875,-1.580903,273.132721,0.898143,0.2980,0.001997,13.805145,7.073290,5.929761,0.946143
2003-01-15,-59.5,1.5,0.0,0.590822,273.192474,259.073090,273.192474,277.799255,286.895844,85.665604,...,-13.840185,-0.886921,273.075989,-0.020427,0.2080,0.002030,13.703369,6.971072,5.808716,0.940503
2003-01-15,-59.5,2.5,0.0,0.556442,273.239288,259.029022,273.239288,277.803558,286.847046,85.945564,...,-13.721111,-0.987361,273.111450,-0.006477,0.1540,0.002075,13.607758,6.874149,5.685569,0.935498
2003-01-15,-59.5,3.5,0.0,0.556442,273.258575,258.989075,273.258575,277.851196,286.802826,86.460487,...,-12.907546,-0.686019,273.103760,0.069008,0.0855,0.002108,13.544250,6.813395,5.606212,0.932689
2003-01-15,-59.5,4.5,0.0,0.584355,273.276489,258.946411,273.276489,277.882935,286.755585,86.692123,...,-12.451134,-0.850394,273.129791,-1.276983,0.0880,0.002122,13.479095,6.751677,5.536650,0.929952


In [5]:
df = df[df.lsm == 0]
df.to_pickle("~/work/LCC_pred/data/data_monthly.pkl")
#df.to_csv("~/work/AOSC647/data/data_monthly.csv")
